Code to understand the Duke-WLOA AMD dataset better 

- understand reading Image annotations. Otherwise explore reading .mat files
- how converting image data into numpy arrays work
- how to generate embeddings
    - which data type to use for this task
    - which model to use?
    - what are the steps for generating image embeddings
    - how does the model architecture look like for this task
- Write the code
- Store embeddings as pytorch arrays




In [2]:
import cv2
import numpy as np

In [3]:
# Load the image
image_path = '/home/suraj/Data/Duke_WLOA_RL_Annotated/AMD_PNG/layer/AMD_001_024.png'
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print("Error: Could not load the image. Check the file path.")
    exit()

In [4]:
# Convert the image from BGR (OpenCV default) to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [5]:
# Get unique colors in the image
unique_colors = np.unique(image_rgb.reshape(-1, image_rgb.shape[2]), axis=0)

In [6]:
# Filter out grayscale colors (where R ≈ G ≈ B) to focus on annotations
colored_annotations = [color for color in unique_colors if not (abs(color[0] - color[1]) < 10 and abs(color[1] - color[2]) < 10)]

/tmp/ipykernel_50861/1291825849.py:2: RuntimeWarning: overflow encountered in scalar subtract
  colored_annotations = [color for color in unique_colors if not (abs(color[0] - color[1]) < 10 and abs(color[1] - color[2]) < 10)]


In [7]:
# Group similar colors into 3 clusters based on RGB proximity (simple tolerance)
tolerance = 20  # Adjust this value if needed
main_colors = []
for color in colored_annotations:
    matched = False
    for main_color in main_colors:
        if (abs(color[0] - main_color[0]) < tolerance and
            abs(color[1] - main_color[1]) < tolerance and
            abs(color[2] - main_color[2]) < tolerance):
            matched = True
            break
    if not matched:
        main_colors.append(color)

/tmp/ipykernel_50861/3331092312.py:8: RuntimeWarning: overflow encountered in scalar subtract
  abs(color[1] - main_color[1]) < tolerance and
/tmp/ipykernel_50861/3331092312.py:9: RuntimeWarning: overflow encountered in scalar subtract
  abs(color[2] - main_color[2]) < tolerance):


In [8]:
# Limit to 3 main colors (assuming 3 layers)
main_colors = main_colors[:3]

In [10]:
# Print the color codes (RGB and Hex) for the 3 main annotations
print("Detected colored annotations (3 main layers):")
for i, color in enumerate(main_colors, 1):
    rgb = (color[0], color[1], color[2])
    hex_code = '#{:02x}{:02x}{:02x}'.format(color[0], color[1], color[2])
    print(f"Layer {i}: RGB = {rgb}, Hex = {hex_code}")

Detected colored annotations (3 main layers):
Layer 1: RGB = (np.uint8(0), np.uint8(128), np.uint8(0)), Hex = #008000
Layer 2: RGB = (np.uint8(3), np.uint8(121), np.uint8(3)), Hex = #037903
Layer 3: RGB = (np.uint8(5), np.uint8(5), np.uint8(242)), Hex = #0505f2


In [11]:
# Define regions for each layer (approximate y-coordinates based on the image)
height = image_rgb.shape[0]
top_layer_y = int(height * 0.3)    # Top third for red layer
mid_layer_y = int(height * 0.5)    # Middle for green layer
bottom_layer_y = int(height * 0.7) # Bottom third for blue layer

# Sample colors from each region (average over a small horizontal strip)
def get_layer_color(y_pos, width_range=50):
    start_x = image_rgb.shape[1] // 2 - width_range // 2
    end_x = start_x + width_range
    strip = image_rgb[y_pos-2:y_pos+2, start_x:end_x]
    return tuple(np.mean(strip, axis=(0, 1)).astype(int))

# Get average colors for each layer
colors = [
    get_layer_color(top_layer_y),    # Top layer (expected red)
    get_layer_color(mid_layer_y),    # Middle layer (expected green)
    get_layer_color(bottom_layer_y)  # Bottom layer (expected blue)
]

# Convert to hex codes
hex_codes = [f'#{color[0]:02x}{color[1]:02x}{color[2]:02x}' for color in colors]

# Print the color codes
print("Detected colored annotations (3 main layers):")
for i, (rgb, hex_code) in enumerate(zip(colors, hex_codes), 1):
    print(f"Layer {i} (expected {'red' if i==1 else 'green' if i==2 else 'blue'}): RGB = {rgb}, Hex = {hex_code}")

Detected colored annotations (3 main layers):
Layer 1 (expected red): RGB = (np.int64(99), np.int64(99), np.int64(99)), Hex = #636363
Layer 2 (expected green): RGB = (np.int64(74), np.int64(74), np.int64(74)), Hex = #4a4a4a
Layer 3 (expected blue): RGB = (np.int64(42), np.int64(42), np.int64(42)), Hex = #2a2a2a
